In [1]:
import pydap.client
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
Start_Year = 2008
End_Year = 2018


In [2]:
url_dict = {}

In [3]:
#not used




# url_dict['WindFromDir']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_'
# url_dict['BurningIndex']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_'
# url_dict['FuelMoist100']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm100/fm100_'
# url_dict['FuelMoist1000']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm1000/fm1000_'
# url_dict['EnergyRelease']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/erc/erc_'
# url_dict['DroughtSeverity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pdsi/pdsi_'
# url_dict['EvapoTranspiration']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/etr/etr_'
# url_dict['EvapoTranspir_G']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pet/pet_'
# url_dict['VaporPressDeficit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vpd/vpd_'

In [4]:
# completed:
# url_dict['WindVelocity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_'  #not finished,[Errno 2] No such file or directory: 'wind_speed_m/s.npz'

# url_dict['BurningIndex']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_'
# url_dict['Sradiation']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_'
# url_dict['Shumidit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_'
# url_dict['Rhumiditymin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_'
# url_dict['Rhumiditymax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_'
# url_dict['Precip']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
# url_dict['Tempmax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmx/tmmx_'
# url_dict['Tempmin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmn/tmmn_'

In [5]:
base_url_part = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url = base_url_part + str(End_Year) + '.nc'
dataset = pydap.client.open_url(url)

In [6]:
#Slice the Bounding Box we wanted here.
lon_ = dataset['lon']
lon = lon_[:].data
lat_ = dataset['lat']
lat = lat_[:].data

 

#Modified approximately to california state 
import numpy as np
SD_California_state = np.where((lat<42.0) & (lat>34.0))[0]
SD_California_state = np.where((lon<-115.0) & (lon>-125))[0]

lat.shape, lon.shape

((585,), (1386,))

In [7]:
lat_begin, lat_end = SD_California_state[0], SD_California_state[-1]
lon_begin, lon_end = SD_California_state[0], SD_California_state[-1]
lat_begin, lat_end, lon_begin, lon_end 

(0, 234, 0, 234)

In [8]:
class data_object:
    def __init__(self, _values, _days, _lats, _lons, _col_name, _year ):
        self.values = _values
        self.days = _days
        self.lats = _lats
        self.lons = _lons
        self.col_name = _col_name
        self.year = _year 

In [9]:
flag=0
final_df = pd.DataFrame()
out = []
for Year in tqdm(range(Start_Year, End_Year+1)):

    print("Downloading data for year {}".format(Year))
    url = base_url_part + str(Year) + '.nc'
    dataset = pydap.client.open_url(url)
    #Create the Base DataFrame with Days and Co-ordinates
    days = pd.to_timedelta(dataset['day'].data[:], unit='days')
    days = days + pd.to_datetime("19000101", format="%Y%m%d")
    index = pd.MultiIndex.from_product([days.values,
                                        lat[SD_California_state].astype('f8'),
                                        lon[SD_California_state].astype('f8')],
                                       names=['date', 'latitude', 'longitude'])
    base_df = pd.DataFrame(index=index) 
    ##Load each Attriubute below
    print()
    for key, item in tqdm(url_dict.items()):
        print ("Processing for", item)
        url = item + str(Year) + '.nc' 
        dataset = pydap.client.open_url(url)
        
        for column in dataset.keys():
            if column in ('lat', 'lon', 'crs', 'day'):
                continue
            #Here is where the Data Download happens
            attrs = dataset[column].attributes
            missing_value, scale, bias = attrs['missing_value'], attrs['scale_factor'], attrs['add_offset']
            print("'{}': missing_values: {}, scale: {}, bias: {}".format(column, missing_value, scale, bias))
            raw_data = dataset[column][:, lat_begin:lat_end, lon_begin:lon_end]

            base_date = pd.to_datetime("19000101", format="%Y%m%d")
            values, days, lats, lons = raw_data.data

            print("Data downloaded")
            days = base_date + pd.to_timedelta(days, unit="days")
            col_name = column + "_" + dataset[column].units
            col_name = col_name.replace("/","") #remove the / so, the filename for npz is regconized 
            if key == 'Tempmax':
                col_name = "max_" + col_name
            elif key == 'Tempmin':
                col_name = "min_" + col_name
            values = values.astype('d') # Double-precision floating-point number type, compatible with Python float and C double.
            values[values == missing_value] = np.nan
            values = values * scale + bias
            print(f"values shape: {values.shape}" )
            print(f"days shape: {days.shape}" )
            print(f"lats shape: {lats.shape}" )
            print(f"lons shape:  {lons.shape}")
            print(f"col_name: {col_name}" )
            print(f"year: {Year}" )
            
            out.append(data_object(values, days, lats, lons, col_name, Year))   
            print(f"len {len(out)}")

    np.savez_compressed(f"{col_name}", data=out)
    print(f"{col_name}_ saved")
#             print(f'the type of values.ravel() is: {type(values.ravel())}') # debugging
#             print(f'the value of values.ravel() is: {values.ravel()}') # debugging
#             base_df.loc[(days, lats, lons), col_name] = values.ravel()  # ravel()- Return a contiguous flattened array
#     display(base_df.head(3))
#     base_df.to_parquet("gridMet_{}.parquet.gz".format(Year), compression='gzip')
#     final_df = pd.concat([final_df, base_df])

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_
'wind_speed': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
values shape: (366, 234, 234)
days shape: (366,)
lats shape: (234,)
lons shape:  (234,)
col_name: wind_speed_ms
year: 2008
len 1
wind_speed_ms_ saved



  0%|          | 0/1 [00:00<?, ?it/s]

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_
'wind_speed': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
values shape: (365, 234, 234)
days shape: (365,)
lats shape: (234,)
lons shape:  (234,)
col_name: wind_speed_ms
year: 2009
len 2
wind_speed_ms_ saved



...

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_
'wind_speed': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
values shape: (366, 234, 234)
days shape: (366,)
lats shape: (234,)
lons shape:  (234,)
col_name: wind_speed_ms
year: 2016
len 9
wind_speed_ms_ saved



  0%|          | 0/1 [00:00<?, ?it/s]

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_
'wind_speed': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
values shape: (365, 234, 234)
days shape: (365,)
lats shape: (234,)
lons shape:  (234,)
col_name: wind_speed_ms
year: 2017
len 10
wind_speed_ms_ saved



  0%|          | 0/1 [00:00<?, ?it/s]

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_
'wind_speed': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
values shape: (365, 234, 234)
days shape: (365,)
lats shape: (234,)
lons shape:  (234,)
col_name: wind_speed_ms
year: 2018
len 11
wind_speed_ms_ saved


In [10]:
# base_df